In [13]:
import numpy as np
import pandas as pd
import pickle


books = pd.read_csv('./dataset/books.csv')
ratings = pd.read_csv('./dataset/ratings.csv')


In [14]:
books = books[['id','book_id', 'authors', 'original_title', 'average_rating', 'image_url', 'ratings_count']]

books.dropna(inplace=True)


In [15]:
books['popularity'] = books['average_rating'] * books['ratings_count']

In [16]:
books['info'] = books['authors'] + books['original_title']


In [17]:
# Ratings modified for merging

ratings = pd.merge(books, ratings)
ratings = ratings[['original_title', 'user_id', 'rating', 'popularity', 'book_id', 'image_url']]

### Vectorize the inputs here 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer (max_features=5000, stop_words='english')


In [19]:
vector = cv.fit_transform(books['info']).toarray()
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'ワンダーランド', '春樹1q84', '村上'], dtype=object)

In [20]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)

## Main function

In [22]:
# Similar books retrieval
def similar_books(book):
    book_index = books[books['original_title'] == book].index[0]
    print('Book index', book_index)
    distances = similarity[book_index]
    print(distances)
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x : x[1])[:5]
    # print(movies_list)
    for i in movies_list:
        print(books.iloc[i[0]].original_title)
# Popular books
def popularity_based_rec(df, group_col, rating_col):
    grouped = df.groupby(group_col).agg({rating_col: [np.size, np.sum, np.mean]})
    popular = grouped.sort_values((rating_col, "mean"), ascending = False)
    total_sum = grouped[rating_col]["sum"].sum()
    popular["percentage"] = popular[rating_col]["sum"].div(total_sum) * 100

    return popular.sort_values(("percentage"), ascending = False)[:10]

In [31]:
popularity_stats = popularity_based_rec(ratings, "original_title", "popularity")
# print(popularity_stats.shape)
popularity_stats.head()

(10, 4)


C:\Users\Bemnet Aschalew\AppData\Local\Temp\ipykernel_1652\2802349428.py:14: FutureWarning: The provided callable <function sum at 0x00000227F15544A0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  grouped = df.groupby(group_col).agg({rating_col: [np.size, np.sum, np.mean]})
C:\Users\Bemnet Aschalew\AppData\Local\Temp\ipykernel_1652\2802349428.py:14: FutureWarning: The provided callable <function mean at 0x00000227F1555580> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped = df.groupby(group_col).agg({rating_col: [np.size, np.sum, np.mean]})


In [24]:
pickle.dump(books.to_dict(), open('./dataset/books_dict.pkl','wb'))
pickle.dump(similarity,open('./dataset/similarity.pkl','wb'))
pickle.dump(ratings.to_dict(), open('./dataset/popularity.pkl', 'wb'))